<img src="./img/header.png">
# Globus SDK
https://github.com/globusonline/globus-sdk-python


# Globus SDK Docs
http://globusonline.github.io/globus-sdk-python/


# Globus CLI
https://github.com/globusonline/globus-cli

### Some endpoints and associated endpoint IDs for the tutorial
go#ep1: ddb59aef-6d04-11e5-ba46-22000b92c6ec <br>
go#ep2: ddb59af0-6d04-11e5-ba46-22000b92c6ec


In [1]:
from __future__ import print_function # for python 2
from globus_sdk import TransferClient

tutorial_endpoints = {"go#ep1":"ddb59aef-6d04-11e5-ba46-22000b92c6ec",
                       "go#ep2":"ddb59af0-6d04-11e5-ba46-22000b92c6ec"}

# Transfer Client

## Instantiate Transfer Client

In [2]:
tc = TransferClient() # uses transfer_token from the config file

## Low-level Interface

In [3]:
print("==== My Endpoints ====")
r = tc.get("/endpoint_search?filter_scope=my-endpoints")
for epdict in r.data["DATA"]:
    print("Display Name:%s\nID:%s\n"%(epdict["display_name"], epdict["id"]))

==== My Endpoints ====
Display Name:air
ID:6f67020c-f22d-11e5-9839-22000b9da45e

Display Name:ben_test
ID:07e41dd4-a5ae-11e5-99c1-22000b96db58

Display Name:blaiszik-share
ID:07256404-d9aa-11e5-976c-22000b9da45e

Display Name:None
ID:cabace53-6d04-11e5-ba46-22000b92c6ec

Display Name:None
ID:c12cf6f0-6d04-11e5-ba46-22000b92c6ec

Display Name:None
ID:db57dce6-6d04-11e5-ba46-22000b92c6ec

Display Name:None
ID:c8ee7e5c-6d04-11e5-ba46-22000b92c6ec

Display Name:None
ID:a1caca84-7745-11e5-9924-22000b96db58

Display Name:None
ID:5bc71728-9ad0-11e5-9997-22000b96db58

Display Name:None
ID:da434408-be18-11e5-9a0e-22000b96db58



## High-level Interface
provides iterators for list responses

In [4]:
print("==== My Endpoints ====")
for r in tc.endpoint_search(filter_scope='my-endpoints'):
    print("Display Name:%s\nID:%s\n"%(r.data["display_name"], r.data["id"]))

==== My Endpoints ====
Display Name:air
ID:6f67020c-f22d-11e5-9839-22000b9da45e

Display Name:ben_test
ID:07e41dd4-a5ae-11e5-99c1-22000b96db58

Display Name:blaiszik-share
ID:07256404-d9aa-11e5-976c-22000b9da45e

Display Name:None
ID:cabace53-6d04-11e5-ba46-22000b92c6ec

Display Name:None
ID:c12cf6f0-6d04-11e5-ba46-22000b92c6ec

Display Name:None
ID:db57dce6-6d04-11e5-ba46-22000b92c6ec

Display Name:None
ID:c8ee7e5c-6d04-11e5-ba46-22000b92c6ec

Display Name:None
ID:a1caca84-7745-11e5-9924-22000b96db58

Display Name:None
ID:5bc71728-9ad0-11e5-9997-22000b96db58

Display Name:None
ID:da434408-be18-11e5-9a0e-22000b96db58



## Endpoint search by name

In [5]:
search_str = "go#ep2"
print("==== Displaying endpoint matches for search: '%s' ===="%search_str)
for r in tc.endpoint_search(search_str):
    print("Display Name:%s\nID:%s\n"%(r.data["display_name"], r.data["id"]))

==== Displaying endpoint matches for search: 'go#ep2' ====
Display Name:Globus Tutorial Endpoint 2
ID:ddb59af0-6d04-11e5-ba46-22000b92c6ec



# File operations

### Path encoding & UTF gotchas

### Autoactivate Endpoints
Here we activate the tutorial endpoints go#ep1 and go#ep2 using their endpoint_ids

In [6]:
#help(tc.endpoint_activate)
r_go1 = tc.endpoint_autoactivate(tutorial_endpoints['go#ep1'])
r_go2 = tc.endpoint_autoactivate(tutorial_endpoints['go#ep2'])

### Get a directory listing

In [7]:
# help(tc.operation_ls)
endpoint_id = tutorial_endpoints['go#ep1'] 
endpoint_path = '/~/shared_dir/'
r = tc.operation_ls(endpoint_id,path=endpoint_path)
print("==== Endpoint_ls for endpoint %s %s ===="%(endpoint_id,endpoint_path))
for item in r.data['DATA']:
    print("%s: %s [%s]"%(item['type'],item['name'],item['size']))

==== Endpoint_ls for endpoint ddb59aef-6d04-11e5-ba46-22000b92c6ec /~/shared_dir/ ====
dir: new_dir [4096]
file: Ag-microparticles.TIF [1384141]
file: ConductivityRestoration.png [4499617]
file: CuSn_battery_electrode-before.mov [3702784]


### Make directory

In [ ]:
#help(tc.operation_mkdir)
endpoint_id = tutorial_endpoints['go#ep1'] 
endpoint_path = '/~/shared_dir/new_dir'
r = tc.operation_mkdir(endpoint_id,path=endpoint_path)

### Rename

In [9]:
#help(tc.operation_rename)
endpoint_id = tutorial_endpoints['go#ep1'] 
endpoint_path = '/~/shared_dir/'
tc.operation_rename(endpoint_id, oldpath="/~/file1.txt",
                    newpath="/~/project1data.txt")

TransferAPIError: (404, u'NotFound', u"Path not found, Error: The source path '~/file1.txt' does not exist\n", u'9iFv4d2jb')

# Task submission

### Transfer w/o sync


In [19]:
# help(tc.submit_transfer)
source_endpoint_id = tutorial_endpoints['go#ep1']
source_path = '/~/shared_dir/'

dest_endpoint_id = tutorial_endpoints['go#ep2']
dest_path = '/~/'

label = "My tutorial transfer"

transfer_items = []

## Recursively transfer source path contents
transfer_items.append(
    tc.make_submit_transfer_item(source_path,
                                 dest_path,
                                 recursive=True))
## Alternatively, transfer a specific file
# transfer_items.append(
#     tc.make_submit_transfer_item("/source/path/file.txt",
#                                  "/dest/path/file.txt"))

transfer_data = tc.make_submit_transfer_data(
    source_endpoint_id,
    dest_endpoint_id,
    transfer_items, label=label)
transfer_result = tc.submit_transfer(transfer_data)
print("task_id = ", transfer_result.data["task_id"])

task_id =  1a09c8a8-001a-11e6-a717-22000bf2d559


# Task management

### Get Task By ID

In [ ]:
r = tc.get_task(transfer_result.data['task_id'])
r.data

### Get task_list

In [36]:
# help(tc.task_list)
r = tc.task_list()
for i,item in enumerate(r):
    print(item.data)
    if r > 10:
        break

{u'files': 3, u'bytes_transferred': 9586542, u'DATA_TYPE': u'task', u'sync_level': None, u'completion_time': u'2016-04-11 19:18:02+00:00', u'canceled_by_admin_message': None, u'deadline': u'2016-04-12 19:18:00+00:00', u'destination_endpoint': u'go#ep2', u'bytes_checksummed': 0, u'delete_destination_extra': False, u'nice_status_details': None, u'source_endpoint_id': u'ddb59aef-6d04-11e5-ba46-22000b92c6ec', u'request_time': u'2016-04-11 19:18:00+00:00', u'nice_status': None, u'subtasks_expired': 0, u'subtasks_canceled': 0, u'label': u'My tutorial transfer', u'faults': 0, u'destination_endpoint_display_name': u'Globus Tutorial Endpoint 2', u'subtasks_total': 6, u'source_endpoint_display_name': u'Globus Tutorial Endpoint 1', u'command': u'API 0.10', u'nice_status_expires_in': None, u'subtask_link': {u'href': u'task/1a09c8a8-001a-11e6-a717-22000bf2d559/subtask_list?format=json', u'resource': u'subtask list', u'rel': u'child', u'DATA_TYPE': u'link', u'title': u'child subtask list'}, u'owner_

### Update task by id (label, deadline)

In [ ]:
help(tc.update_task)

### Cancel task by id

In [ ]:
help(tc.cancel_task)

### Get event list for task

### Get task pause info

# Bookmarks

# Create a Bookmark

In [ ]:
bookmark_name = "My Tutorial Bookmark"
endpoint_id = tutorial_endpoints['go#ep1']
endpoint_path = '/'
r = tc.create_bookmark({"endpoint_id":endpoint_id, "path":endpoint_path,"name":bookmark_name})
r

# Delete a Bookmark

In [ ]:
tc.delete_bookmark(r.data['id'])

### CRUD + list

# Shared endpoints

### Access_manager role

### Get list off access rules

### Get access rule by id

### Create access rule

### Update access rule

### Delete access rule